<a href="https://colab.research.google.com/github/Tclack88/AIMusic/blob/master/AIGenerateMusicipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LambdaCallback

In [0]:
us!wget https://raw.githubusercontent.com/Tclack88/Machine-Learning-Projects/master/GenerateMusic_NeuralNetwork_Incomplete/master_songs.enc


--2019-12-27 18:58:06--  https://raw.githubusercontent.com/Tclack88/Machine-Learning-Projects/master/GenerateMusic_NeuralNetwork_Incomplete/master_songs.enc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246433 (241K) [text/plain]
Saving to: ‘master_songs.enc’

master_songs.enc    100%[===================>] 240.66K  --.-KB/s    in 0.05s   

2019-12-27 18:58:08 (4.75 MB/s) - ‘master_songs.enc’ saved [246433/246433]



In [0]:
filename = "master_songs.enc"
raw_text = open(filename).read()

In [0]:
chars = list(set(raw_text))

int_char = {i:c for i,c in enumerate(chars)}
char_int = {c:i for i,c in enumerate(chars)}

In [0]:
max_len = 60
step = 5

encoded = [char_int[c] for c in raw_text]

sequences = []
next_chars = []

for i in range(0, len(encoded) - max_len, step):
  sequences.append(iencoded[i : i + max_len])
  next_chars.append(encoded[i + max_len])

In [0]:
X = np.zeros((len(sequences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
  for s, char in enumerate(sequence):
    X[i, s, char] = 1
  y[i, next_chars[i]] = 1

In [0]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer=RMSprop(learning_rate=.01)

model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
import sys
import random

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(raw_text) - max_len - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = raw_text[start_index: start_index + max_len]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(3000):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
!ls

master_songs.enc  sample_data


## Good attempt below, do not erase (Model weights not saved :/)

In [0]:
model.fit(X, y, batch_size=512, epochs=5, callbacks=[print_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 49275 samples
Epoch 1/5
49152/49275 [============================>.] - ETA: 0s - loss: 1.1710
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "NTW`                          !DN  |HK                      "
NTW`                          !DN  |HK                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [0]:
model.save_weights('model-10-epochs.h5')

In [0]:
!ls

master_songs.enc  model-10-epochs.h5  sample_data


In [0]:
model.fit(X, y, batch_size=512, epochs=10, callbacks=[print_callback])

Train on 49275 samples
Epoch 1/10
49152/49275 [============================>.] - ETA: 0s - loss: 0.9964
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "              !5  |AHMQ                          !AHMQ  |AHM"
              !5  |AHMQ                          !AHMQ  |AHM                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [0]:
!ls

master_songs.enc  model-10-epochs.h5  sample_data


In [0]:
model.save_weights('model-30-epochs.h5')

In [0]:
model.fit(X, y, batch_size=512, epochs=5, callbacks=[print_callback]) #30 adding 5 (plays C# forever at one point)

Train on 49275 samples
Epoch 1/5
49152/49275 [============================>.] - ETA: 0s - loss: 0.6585
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "KW  |CK        |W    !C  |:      !W  |Q      !:  |?    !Q  |"
KW  |CK        |W    !C  |:      !W  |Q      !:  |?    !Q  |W                    !C  |W                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [0]:
model.save_weights('model-35-epochs.h5')

In [0]:
model.fit(X, y, batch_size=512, epochs=5, callbacks=[print_callback]) #35 -> 40

Train on 49275 samples
Epoch 1/5
49152/49275 [============================>.] - ETA: 0s - loss: 0.4425
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "               !>BGN  |>BGS                          !>BGS  "
               !>BGN  |>BGS                          !>BGS  |>BGS                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [0]:
model.fit(X, y, batch_size=512, epochs=5, callbacks=[print_callback]) #40 -> 45

Train on 49275 samples
Epoch 1/5
49152/49275 [============================>.] - ETA: 0s - loss: 0.2870
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "    !=I  |9E                  !9E  |?K                  !?K "
    !=I  |9E                  !9E  |?K                  !?K  |@L                  !@U  |CFKOX                                                  |CKO                              |CF                          |CF                              |:DJP                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [0]:
 model.fit(X, y, batch_size=512, epochs=5, callbacks=[print_callback]) #45 -> 50

Train on 49275 samples
Epoch 1/5
49152/49275 [============================>.] - ETA: 0s - loss: 0.2007
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " !(4JLPS  |-9ILQ                                            "
 !(4JLPS  |-9ILQ                                                !EIN  |CFKOR                                                !E  |F                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                          |FFRR                        |FF  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


     !T  |P        !P  |O        !P  |S        !PT  |0Z        !K  |Q        !M  |K        !?S  |O        |[        !CO  |LO        |X      !U  |O        !CO  |LO              !O  |Y                    !MQY  |N                                                   !.::HQ  |:DM                          !?FP  |:AFIMP                          !:AR  |:DMT                          !AFIMRW  |:                          !AHMQ  |AHMQ                          !AHY  |AHM                          !A  |3KRR                          !?KPT  |8KPT        !8?HP  |4;              !C[  |=M                  !=MQ  |7Y                    !7  |EH                  !EM OTW  |NQV                                    !7QT\  |CHKOR              !CT  |LS                    !-Z  |DGMSV                                                !CGKO  |:AFJM                          !AFHMR                          !6  |AFJMR                          !AJM  |AHMQ                          !AEY  |3W                          !?FY  |R     

In [0]:
model.save_weights('model-50-epochs.h5')

## Restored weights from previous session

In [0]:
model.load_weights('model-50-epochs.h5')

In [0]:
 model.fit(X, y, batch_size=512, epochs=5, callbacks=[print_callback]) #50 -> 55

Train on 49275 samples
Epoch 1/5
49152/49275 [============================>.] - ETA: 0s - loss: 0.8339
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "    !GV  |Eb        !E  |G        !Gb  |D_        !D  |E    "
    !GV  |Eb        !E  |G        !Gb  |D_        !D  |E        !BNNRU  |5NNNUUNNUUWNNUUWNNUUWNNNUUeURUUWNUU  |UUNNUUNNUU                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   